# Fire and Flood Risks in US

In this project we will evaluate Fire and Flood claims in the United States. We will determine key metrics based on data available related to frequency and severity of claims, average dollar amount of claims, risk of claims, loss ratios and as a bonus: investment income from surplus of premiums. 

In [84]:
#Import libraries
import numpy as np
import pandas as pd
from pathlib import Path
import hvplot.pandas

%matplotlib inline

## Data Cleaning

We will read, clean and combine the following CSV files for further analysis:
1. Nonres_Fire_Dollr_Loss_by_Cause.csv
2. Res_Bld_Fire_Dollr_Est_by_Cause.csv

In [85]:
#Set Path using pathlib
nonres_fire_losses_csv = Path(
    "fire_loss_estimates/Nonres_Fire_Dollr_Loss_by_Cause.csv")
res_fire_losses_csv = Path(
    "fire_loss_estimates/Res_Bld_Fire_Dollr_Est_by_Cause.csv")

In [86]:
#Read CSV 'Nonres_Fire_Dollr_Loss_by_Cause.csv' into dataframe
nonres_fire_losses = pd.read_csv(nonres_fire_losses_csv)
nonres_fire_losses.head()

,Years,Intentional,Playing with Heat Source,Smoking,Heating,Cooking,Electrical Malfunction,Appliances,Open Flame,Other Heat,Other Equipment,Natural,Exposure,Equipment Malfunction,"Other Unintentional, Careless",Cause Under Investigation,Total Nonresidential Dollar Loss*,Unnamed: 17,Unnamed: 18
0,2003,"$364,600,000.00","$12,100,000.00","$43,900,000.00","$159,300,000.00","$56,900,000.00","$514,600,000.00","$102,300,000.00","$282,100,000.00","$207,200,000.00","$194,600,000.00","$258,100,000.00","$198,000,000.00","$241,100,000.00","$322,000,000.00","$149,400,000.00","$3,106,000,000.00",NaN,NaN
1,2004,"$324,600,000.00","$12,400,000.00","$26,400,000.00","$118,000,000.00","$70,400,000.00","$403,900,000.00","$58,700,000.00","$280,900,000.00","$173,400,000.00","$300,900,000.00","$170,700,000.00","$119,300,000.00","$224,700,000.00","$452,500,000.00","$98,700,000.00","$2,835,600,000.00",NaN,NaN
2,2005,"$315,300,000.00","$21,100,000.00","$24,800,000.00","$86,600,000.00","$62,200,000.00","$478,600,000.00","$94,000,000.00","$207,200,000.00","$216,200,000.00","$208,600,000.00","$144,600,000.00","$171,300,000.00","$206,500,000.00","$332,600,000.00","$178,300,000.00","$2,747,800,000.00",NaN,NaN
3,2006,"$389,000,000.00","$9,400,000.00","$28,300,000.00","$134,500,000.00","$50,500,000.00","$500,600,000.00","$115,000,000.00","$137,900,000.00","$156,800,000.00","$110,900,000.00","$219,700,000.00","$137,100,000.00","$461,600,000.00","$300,100,000.00","$169,200,000.00","$2,920,500,000.00",NaN,NaN
4,2007,"$529,700,000.00","$11,900,000.00","$37,900,000.00","$88,400,000.00","$51,100,000.00","$488,600,000.00","$75,700,000.00","$274,300,000.00","$275,700,000.00","$235,700,000.00","$247,900,000.00","$235,400,000.00","$276,700,000.00","$455,700,000.00","$188,200,000.00","$3,473,000,000.00",NaN,NaN


In [87]:
#Drop all columns unrelated to total dollar amount of fire damages. 
nonres_fire_losses = nonres_fire_losses[["Years", "Total Nonresidential Dollar Loss*"]]

#Set Index equal to 'Years'.
nonres_fire_losses.set_index("Years", inplace = True)

#Drop all '$' and ','.
nonres_fire_losses["Total Nonresidential Dollar Loss*"] = nonres_fire_losses[
    "Total Nonresidential Dollar Loss*"].str.replace('$','')
nonres_fire_losses["Total Nonresidential Dollar Loss*"] = nonres_fire_losses[
    "Total Nonresidential Dollar Loss*"].str.replace(',','')
nonres_fire_losses.head()

,Total Nonresidential Dollar Loss*
Years,
2003,3106000000.00
2004,2835600000.00
2005,2747800000.00
2006,2920500000.00
2007,3473000000.00


In [88]:
#Read CSV 'Res_Bld_Fire_Dollr_Est_by_Cause.csv' into dataframe
res_fire_losses = pd.read_csv(res_fire_losses_csv)
res_fire_losses.head()

,Years,Intentional,Playing with Heat Source,Smoking,Heating,Cooking,Electrical Malfunction,Appliances,Open Flame,Other Heat,Other Equipment,Natural,Exposure,Equipment Malfunction,"Other Unintentional, Careless",Cause Under Investigation,Total Residential*,Unnamed: 17,Unnamed: 18
0,2003,"705,400,000","167,000,000","337,100,000","461,200,000","351,500,000","1,200,100,000","304,100,000","910,000,000","691,300,000","133,400,000","435,900,000","272,100,000","503,200,000","1,057,000,000","233,600,000","7,762,800,000",NaN,NaN
1,2004,"664,100,000","143,300,000","348,300,000","383,700,000","297,100,000","1,201,700,000","246,200,000","942,400,000","590,800,000","143,800,000","522,900,000","279,200,000","435,200,000","1,032,900,000","272,100,000","7,503,500,000",NaN,NaN
2,2005,"727,100,000","162,800,000","392,600,000","379,800,000","343,500,000","1,450,000,000","263,100,000","1,051,900,000","664,400,000","167,500,000","454,100,000","303,300,000","513,300,000","1,061,900,000","418,000,000","8,353,200,000",NaN,NaN
3,2006,"667,100,000","141,200,000","402,100,000","390,500,000","300,600,000","1,270,700,000","290,700,000","894,000,000","1,026,400,000","125,700,000","520,500,000","289,000,000","467,800,000","1,115,900,000","375,300,000","8,277,500,000",NaN,NaN
4,2007,"584,500,000","113,000,000","320,500,000","314,000,000","277,200,000","1,229,300,000","240,400,000","798,800,000","503,800,000","146,600,000","438,100,000","1,914,500,000","408,300,000","1,047,700,000","331,200,000","8,667,900,000",NaN,NaN


In [89]:
#Drop all columns unrelated to total dollar amount of fire damages.
res_fire_losses = res_fire_losses[["Years", "Total Residential*"]]

#Set Index equal to 'Years'.
res_fire_losses.set_index("Years", inplace = True)

#Drop all ','.
res_fire_losses["Total Residential*"] = res_fire_losses[
    "Total Residential*"].str.replace(',','')
res_fire_losses.head()

,Total Residential*
Years,
2003,7762800000
2004,7503500000
2005,8353200000
2006,8277500000
2007,8667900000


In [90]:
#Check Data types for Non-residential fire dollar losses. 
nonres_fire_losses.dtypes

Total Nonresidential Dollar Loss*    object
dtype: object

In [91]:
#Check Data types Residential fire dollar losses. 
res_fire_losses.dtypes

Total Residential*    object
dtype: object

In [92]:
#Convert to float 
nonres_fire_losses['Total Nonresidential Dollar Loss*'] = nonres_fire_losses[
    'Total Nonresidential Dollar Loss*'].astype('float')
nonres_fire_losses['Total Nonresidential Dollar Loss*'].dtype

dtype('float64')

In [93]:
#Convert to float
res_fire_losses["Total Residential*"] = res_fire_losses[
    "Total Residential*"].astype('float')
res_fire_losses["Total Residential*"].dtype

dtype('float64')

# Plot individual and combined dataframes 

In [94]:
#Plot total dollar losses for fires over the past 15 years both Residential and Nonresidential.
res_fire_losses_plot = res_fire_losses.hvplot(title = "Residential Total Fire Dollar Losses")
nonres_fire_losses_plot = nonres_fire_losses.hvplot(title = "Nonresidential Total Fire Dollar Losses")

In [95]:
## Combine Nonresidential and Residential Total Dollar Losses into one dataframe.
all_fire_losses = pd.concat([nonres_fire_losses, res_fire_losses], axis='columns', join='inner')
all_fire_losses.sort_index(axis=0, ascending=True, inplace=True)
all_fire_losses.tail()

,Total Nonresidential Dollar Loss*,Total Residential*
Years,,
2014,2.732400e+09,7.319200e+09
2015,2.851100e+09,7.521300e+09
2016,2.979500e+09,7.660200e+09
2017,2.785800e+09,7.987400e+09
2018,2.656600e+09,8.194500e+09


In [96]:
#Use hvplot to plot Nonresidential and Residential dollar losses.
all_fire_losses_plot = all_fire_losses.hvplot(title = "Nonresidential and Residential Total Fire Dollar Losses")
all_fire_losses_plot

:NdOverlay   [Variable]
   :Curve   [Years]   (value)

## Data Cleaning

We will read, clean and combine the following CSV files for further analysis:
1. Nonres_Bldg_Fire_Est_by_Cause.csv
2. Res_Bldg_Fire_Est_by_Cause.csv

In [97]:
#Set Path using pathlib
nonres_fire_est_csv = Path(
    "fire_loss_estimates/Nonres_Bldg_Fire_Est_by_Cause.csv")
res_fire_est_csv = Path(
    "fire_loss_estimates/Res_Bldg_Fire_Est_by_Cause.csv")

In [98]:
#Read CSV 'Nonres_Bldg_Fire_Est_by_Cause.csv' into dataframe
nonres_fire_est = pd.read_csv(nonres_fire_est_csv)
nonres_fire_est.head()

,Years,Intentional,Playing with Heat Source,Smoking,Heating,Cooking,Electrical Malfunction,Appliances,Open Flame,Other Heat,Other Equipment,Natural,Exposure,Equipment Malfunction,"Other Unintentional, Careless",Cause Under Investigation,Total Nonresidential*,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,2003,"11,300",600,"2,400","10,100","23,900","10,100","2,700","7,500","7,500","4,300","3,800","4,700","6,000","7,500",900,"103,200",NaN,NaN,NaN
1,2004,"10,900",500,"2,100","9,400","25,600","9,800","2,700","7,300","7,300","4,600","3,500","4,100","5,700","7,300",900,"101,900",NaN,NaN,NaN
2,2005,"11,800",500,"2,800","8,200","25,200","10,000","2,800","7,000","7,100","4,200","3,500","4,300","5,300","7,400","1,100","101,400",NaN,NaN,NaN
3,2006,"10,800",600,"2,700","7,500","26,000","9,400","2,700","6,500","6,300","4,000","3,900","4,800","5,000","7,500","1,100","98,900",NaN,NaN,NaN
4,2007,"11,400",500,"2,600","8,500","26,400","9,700","2,900","6,500","6,100","4,400","3,900","4,900","5,100","8,800","1,200","103,000",NaN,NaN,NaN


In [99]:
#Drop all columns unrelated to total fire damages. 
nonres_fire_est = nonres_fire_est[["Years", "Total Nonresidential*"]]

#Set Index equal to 'Years'.
nonres_fire_est.set_index("Years", inplace = True)

#Drop all commas.
nonres_fire_est["Total Nonresidential*"] = nonres_fire_est[
    "Total Nonresidential*"].str.replace(',','')
nonres_fire_est.head()

,Total Nonresidential*
Years,
2003,103200
2004,101900
2005,101400
2006,98900
2007,103000


In [100]:
#Check data types
nonres_fire_est.dtypes

Total Nonresidential*    object
dtype: object

In [101]:
#Convert to 'float'
nonres_fire_est['Total Nonresidential*'] = nonres_fire_est[
    'Total Nonresidential*'].astype('float')
nonres_fire_est['Total Nonresidential*'].dtype

dtype('float64')

In [102]:
#Use hvplot to plot the total amount of claims per year for nonresidential fires.
nonres_fire_est_plot = nonres_fire_est.hvplot(title = "Nonresidential Claims Per Year")

In [103]:
#Read CSV 'Res_Bldg_Fire_Est_by_Cause.csv' into dataframe
res_fire_est = pd.read_csv(res_fire_est_csv)
res_fire_est.head()

,Years,Intentional,Playing with Heat Source,Smoking,Heating,Cooking,Electrical Malfunction,Appliances,Open Flame,Other Heat,Other Equipment,Natural,Exposure,Equipment Malfunction,"Other Unintentional, Careless",Cause Under Investigation,Total Residential*
0,2003,"17,400","3,500","8,900","61,100","153,000","26,400","9,200","23,700","17,900","6,000","6,700","7,000","16,100","22,000","2,200","381,200"
1,2004,"17,500","3,100","9,000","60,600","162,900","27,000","8,600","23,100","17,800","5,600","6,700","7,100","16,000","22,300","2,400","389,800"
2,2005,"18,000","2,900","8,700","54,200","155,500","28,500","8,500","22,900","17,600","5,000","6,700","7,500","15,100","22,500","2,900","376,500"
3,2006,"18,100","3,200","9,700","53,600","168,400","30,000","9,700","22,300","16,100","5,000","7,200","7,900","14,700","23,400","3,300","392,700"
4,2007,"19,000","3,100","8,900","54,400","161,700","30,600","10,200","20,900","15,400","4,600","6,900","10,300","15,200","25,400","3,800","390,300"


In [104]:
#Drop all columns unrelated to total fire damages. 
res_fire_est = res_fire_est[["Years", "Total Residential*"]]

#Set Index equal to 'Years'.
res_fire_est.set_index("Years", inplace = True)

#Drop all commas.
res_fire_est["Total Residential*"] = res_fire_est[
    "Total Residential*"].str.replace(',','')
res_fire_est.tail()

,Total Residential*
Years,
2014,379500
2015,380900
2016,364300
2017,371500
2018,379600


In [105]:
#Check datatypes
res_fire_est.dtypes

Total Residential*    object
dtype: object

In [106]:
#Convert to 'float'
res_fire_est['Total Residential*'] = res_fire_est[
    'Total Residential*'].astype('float')
res_fire_est['Total Residential*'].dtype

dtype('float64')

In [107]:
#Use hvplot to plot the total amount of claims per year for residential fires.
res_fire_est_plot = res_fire_est.hvplot(title = "Residnetial Claims Per Year")

In [128]:
## Combine Nonresidential and Residential Total Claims into one dataframe.
all_fire_claims = pd.concat([nonres_fire_est, res_fire_est], axis='columns', join='inner')
all_fire_claims.sort_index(axis=0, ascending=True, inplace=True)
all_fire_claims.head()

,Total Nonresidential*,Total Residential*
Years,,
2003,103200.0,381200.0
2004,101900.0,389800.0
2005,101400.0,376500.0
2006,98900.0,392700.0
2007,103000.0,390300.0


In [117]:
#Use hvplot to plot Nonresidential and Residential dollar losses.
all_fire_claims_plot = all_fire_claims.hvplot(title = "Nonresidential and Residential Total Fire Claims")
all_fire_claims_plot

:NdOverlay   [Variable]
   :Curve   [Years]   (value)

# Average Claims and Premiums

### A) Calculate average cost per claim, then plot the averages per year. 

In [110]:
#Calculate Nonresidential average cost per claim.
nonres_average_cost_per_claim = (nonres_fire_losses["Total Nonresidential Dollar Loss*"] / 
                                 nonres_fire_est["Total Nonresidential*"])
nonres_average_cost_per_claim.tail(1)

Years
2018    25642.857143
dtype: float64

In [111]:
#Calculate Residential average cost per claim.
res_average_cost_per_claim = (res_fire_losses["Total Residential*"] / 
                              res_fire_est["Total Residential*"])
res_average_cost_per_claim.tail(1)

Years
2018    21587.19705
Name: Total Residential*, dtype: float64

In [112]:
#Use hvplot to plot the average residential and nonresidential average cost per claim
nonres_avg_cost_plot = nonres_average_cost_per_claim.hvplot(title = "Nonresidential Average Cost Per Claim")
res_avg_cost_plot = res_average_cost_per_claim.hvplot(title = "Residential Average Cost Per Claim")

In [129]:
all_average_claims = pd.concat([nonres_average_cost_per_claim, 
                                res_average_cost_per_claim], 
                               axis='columns', join='inner')
all_average_claims.sort_index(axis=0, ascending=True, inplace=True)
all_average_claims.tail(1).round()

,0,Total Residential*
Years,,
2018,25643.0,21587.0


In [130]:
#Use hvplot to plot Nonresidential and Residential dollar losses.
all_average_claims_plot = all_average_claims.hvplot(ylim=(15000, 45000), title = "Nonresidential and Residential Average Fire Cost")
all_average_claims_plot

:NdOverlay   [Variable]
   :Curve   [Years]   (value)

### B) Calculate average premium needed to achieve 20% profit (assume an average of 80% is paid to claims) 

In [65]:
#Set profit variable
nonres_profit = .20
#Take average claim cost and multiply by % profit then add to average cost
nonres_premium = ((nonres_average_cost_per_claim * nonres_profit) + nonres_average_cost_per_claim)
nonres_premium_plot = nonres_premium.hvplot(title = "Nonresidential Average Fire Premium")

In [66]:
#Set Profit variable
res_profit = .20
#Take average claim cost and multiply by % profit then add to average claim cost
res_premium = ((res_average_cost_per_claim * res_profit) + res_average_cost_per_claim)
res_premium_plot = res_premium.hvplot(title = "Residential Average Fire Premium")

In [67]:
#Concat Nonresidential and Residential average Premiums
all_average_premium = pd.concat([nonres_premium, 
                                res_premium], 
                               axis='columns', join='inner')
all_average_premium.sort_index(axis=0, ascending=True, inplace=True)
all_average_premium_plot = all_average_premium.hvplot(title = "Nonresidential and Residential Average Fire Premiums")
all_average_premium_plot

:NdOverlay   [Variable]
   :Curve   [Years]   (value)

# Build Fire Analysis Dashboard

In [68]:
#Create dashboard for 'Nonresidential and Residential Total Fire Dollar Losses', 
#'Nonresidential and Residential Total Fire Claims', 
#'Nonresidential and Residential Average Fire Cost', 
#'Nonresidential and Residential Average Fire Premium' as one tab named "Fire Analysis"

# Part 2 -- Portfolio Return
Given a 20% surplus from premiums collected. Perform a risk return analysis on individual stock or portfolio of stocks. Set "initial_investment" equal to 20% of premiums calulated from above. 

In [135]:
nonres_initital_investment = nonres_average_cost_per_claim * nonres_profit
nonres_initital_investment.to_csv("nonres_initial_investment")
nonres_initital_investment

Years
2003    6019.379845
2004    5565.456330
2005    5419.723866
2006    5905.965622
2007    6743.689320
2008    8293.099897
2009    7241.928251
2010    6512.603062
2011    6367.681499
2012    5824.353448
2013    5663.180363
2014    5492.261307
2015    5451.434034
2016    6155.991736
2017    5019.459459
2018    5128.571429
dtype: float64

In [134]:
res_initial_investment = res_average_cost_per_claim * res_profit
res_initial_investment.to_csv("res_initial_investment")
res_initial_investment

Years
2003    4072.822665
2004    3849.923037
2005    4437.290837
2006    4215.686275
2007    4441.660261
2008    4947.382337
2009    4771.083661
2010    4227.782381
2011    4074.128944
2012    4154.278075
2013    3897.712332
2014    3857.285903
2015    3949.225519
2016    4205.435081
2017    4300.080754
2018    4317.439410
Name: Total Residential*, dtype: float64